In [ ]:
import pandas as pd
import re
# Load the data
file_path = 'drive/MyDrive/filtered.tsv'
data = pd.read_csv(file_path, delimiter='\t', encoding='utf-8')

# Display the first few rows of the dataframe
data.head()


,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


In [ ]:
data.drop('Unnamed: 0', axis=1, inplace=True)


In [ ]:
data["reference"]

0         If Alkar is flooding her with psychic waste, t...
1                                 Now you're getting nasty.
2                  Well, we could spare your life, for one.
3                 Ah! Monkey, you've got to snap out of it.
4                          I've got orders to put her down.
                                ...                        
577772    You didn't know that Estelle had stolen some f...
577773                      It'il suck the life out of you!
577774                     I can't fuckin' take that, bruv.
577775    They called me a fucking hero. The truth is I ...
577776                                 I did not screw him.
Name: reference, Length: 577777, dtype: object

In [ ]:
# Identify rows where the translation is less toxic than the reference
swap_indices = data['trn_tox'] < data['ref_tox']

# Create temporary columns for swapping
data['temp_reference'] = data['reference']
data['temp_translation'] = data['translation']
data['temp_ref_tox'] = data['ref_tox']
data['temp_trn_tox'] = data['trn_tox']

# Perform the swap
data.loc[swap_indices, 'reference'] = data.loc[swap_indices, 'temp_translation']
data.loc[swap_indices, 'translation'] = data.loc[swap_indices, 'temp_reference']
data.loc[swap_indices, 'ref_tox'] = data.loc[swap_indices, 'temp_trn_tox']
data.loc[swap_indices, 'trn_tox'] = data.loc[swap_indices, 'temp_ref_tox']

# Drop the temporary columns
data.drop(['temp_reference', 'temp_translation', 'temp_ref_tox', 'temp_trn_tox'], axis=1, inplace=True)


In [ ]:
data["reference"]

0         If Alkar is flooding her with psychic waste, t...
1                                 Now you're getting nasty.
2                  Well, we could spare your life, for one.
3                 Ah! Monkey, you've got to snap out of it.
4                          I've got orders to put her down.
                                ...                        
577772    You didn't know that Estelle had stolen some f...
577773                    you'd be sucked out of your life!
577774                            I really can't take this.
577775           they said I was a hero, but I didn't care.
577776                                 I did not screw him.
Name: reference, Length: 577777, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)

# Fit and transform the reference text
tfidf_matrix_ref = vectorizer.fit_transform(data['reference'])

# Transform the translation text
tfidf_matrix_trn = vectorizer.transform(data['translation'])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# Save the processed dataset to a TSV file
processed_file_path = 'drive/MyDrive/processed_data.tsv'  # or choose your own file path
data.to_csv(processed_file_path, sep='\t', index=False)
